## Lab 5: Tensorflow Intro


#### CSC 180  Intelligent Systems (Spring 2020)

#### Dr. Haiquan Chen, California State University, Sacramento

# Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)



# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Classification or Regression

Neural networks can function in *** classification or regression***:

* **Regression** - You expect a number as your neural network's prediction.
* **Classification** - You expect a class/category as your neural network's prediction.

The following shows a classification and regression neural network:

![Neural Network Classification and Regression](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_2_ann_class_reg.png "Neural Network Classification and Regression")


Regression networks always have a single output neuron.  Classification neural networks have an output neuron for each class. 

These neurons are grouped into layers:

* **Input Layer** - The input layer accepts feature vectors from the dataset.  Input layers usually have a bias neuron.
* **Output Layer** - The output from the neural network.  The output layer does not have a bias neuron.
* **Hidden Layers** - Layers that occur between the input and output layers.  Each hidden layer will usually have a bias neuron.


# What version of TensorFlow do you have?

TensorFlow is very new and changing rapidly. 

In [2]:
import tensorflow as tf
print("Tensor Flow Version: {}".format(tf.__version__))

Tensor Flow Version: 2.0.0


# Why TensorFlow

* Supported by Google
* Works well on Linux/Mac
* Excellent GPU support
* Most popular today

# Using Keras

[Keras](https://keras.io/) is an official high-end API, supported by Google, for Tensorflow that makes it much easier to create neural networks.    

Keras is a separate install from TensorFlow.  

To install Keras, use **conda install keras** after **conda install tensorflow**.

In [3]:
import tensorflow.keras 
print("Keras Version: {}".format(tensorflow.keras.__version__))

Keras Version: 2.2.4-tf


# Example of TensorFlow Regression: MPG Prediction

This example shows how to encode the MPG dataset for regression.  Notice that:

* Input has both numeric and categorical
* Input has missing values

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

df[0:5]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [5]:
cars = df['name']

df.drop('name',1,inplace=True)

missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

In [6]:
x.shape

(398, 9)

In [7]:
y.shape

(398,)

In [7]:
x

array([[  8., 307., 130., ...,   1.,   0.,   0.],
       [  8., 350., 165., ...,   1.,   0.,   0.],
       [  8., 318., 150., ...,   1.,   0.,   0.],
       ...,
       [  4., 135.,  84., ...,   1.,   0.,   0.],
       [  4., 120.,  79., ...,   1.,   0.,   0.],
       [  4., 119.,  82., ...,   1.,   0.,   0.]], dtype=float32)

In [8]:
y

array([18. , 15. , 18. , 16. , 17. , 15. , 14. , 14. , 14. , 15. , 15. ,
       14. , 15. , 14. , 24. , 22. , 18. , 21. , 27. , 26. , 25. , 24. ,
       25. , 26. , 21. , 10. , 10. , 11. ,  9. , 27. , 28. , 25. , 25. ,
       19. , 16. , 17. , 19. , 18. , 14. , 14. , 14. , 14. , 12. , 13. ,
       13. , 18. , 22. , 19. , 18. , 23. , 28. , 30. , 30. , 31. , 35. ,
       27. , 26. , 24. , 25. , 23. , 20. , 21. , 13. , 14. , 15. , 14. ,
       17. , 11. , 13. , 12. , 13. , 19. , 15. , 13. , 13. , 14. , 18. ,
       22. , 21. , 26. , 22. , 28. , 23. , 28. , 27. , 13. , 14. , 13. ,
       14. , 15. , 12. , 13. , 13. , 14. , 13. , 12. , 13. , 18. , 16. ,
       18. , 18. , 23. , 26. , 11. , 12. , 13. , 12. , 18. , 20. , 21. ,
       22. , 18. , 19. , 21. , 26. , 15. , 16. , 29. , 24. , 20. , 19. ,
       15. , 24. , 20. , 11. , 20. , 21. , 19. , 15. , 31. , 26. , 32. ,
       25. , 16. , 16. , 18. , 16. , 13. , 14. , 14. , 14. , 29. , 26. ,
       26. , 31. , 32. , 28. , 24. , 26. , 24. , 26

In [8]:
model = Sequential()

model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1     #  why input_dim=x.shape[1]?  
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.


W0913 14:15:44.170745 14488 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/100
398/398 - 0s - loss: 169958.3536
Epoch 2/100
398/398 - 0s - loss: 16782.3133
Epoch 3/100
398/398 - 0s - loss: 525.1127
Epoch 4/100
398/398 - 0s - loss: 1268.3984
Epoch 5/100
398/398 - 0s - loss: 341.2692
Epoch 6/100
398/398 - 0s - loss: 100.2554
Epoch 7/100
398/398 - 0s - loss: 97.3362
Epoch 8/100
398/398 - 0s - loss: 75.1442
Epoch 9/100
398/398 - 0s - loss: 74.4091
Epoch 10/100
398/398 - 0s - loss: 72.5188
Epoch 11/100
398/398 - 0s - loss: 71.7518
Epoch 12/100
398/398 - 0s - loss: 71.1975
Epoch 13/100
398/398 - 0s - loss: 70.6378
Epoch 14/100
398/398 - 0s - loss: 70.4709
Epoch 15/100
398/398 - 0s - loss: 69.5546
Epoch 16/100
398/398 - 0s - loss: 68.6421
Epoch 17/100
398/398 - 0s - loss: 68.2349
Epoch 18/100
398/398 - 0s - loss: 67.4454
Epoch 19/100
398/398 - 0s - loss: 67.4187
Epoch 20/100
398/398 - 0s - loss: 66.3955
Epoch 21/100
398/398 - 0s - loss: 65.5446
Epoch 22/100
398/398 - 0s - loss: 65.0612
Epoch 23/100
398/398 - 0s - loss: 64.4131
Epoch 24/100
398/398 - 0s - los

### Monitor the loss at each epoch

One line is produced for each training epoch.  You can eliminate this output by setting the verbose setting of the fit command:

* **verbose=0** - No progress output (use with Juputer if you do not want output)
* **verbose=1** - Display progress bar, does not work well with Jupyter
* **verbose=2** - Summary progress output (use with Jupyter if you want to know the loss at each epoch)


## Use Trained Model to Make Regression Prediction

Next we will perform actual predictions.  These predictions are assigned to the **pred** variable. These are all MPG predictions from the neural network.  

***Notice that the data to predict should be a 2D array!***  

***Notice that the prediction result is also a 2D array!*** 

Neural networks can return multiple values, so the result is always an array.  Here the neural network only returns 1 value per prediction (there are 398 cars, so 398 predictions).  However, a 2D array is needed because the neural network has the potential of returning more than one value. 


In [9]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

Shape: (398, 1)
[[14.295172 ]
 [14.286657 ]
 [14.668158 ]
 [16.209463 ]
 [15.1081295]
 [12.428854 ]
 [12.156668 ]
 [12.6303005]
 [13.063772 ]
 [12.959219 ]
 [10.978979 ]
 [13.490011 ]
 [ 8.106436 ]
 [ 8.265875 ]
 [26.689886 ]
 [19.894506 ]
 [19.762747 ]
 [17.919607 ]
 [26.419027 ]
 [22.35594  ]
 [27.576696 ]
 [26.411703 ]
 [27.73406  ]
 [26.033316 ]
 [18.468847 ]
 [22.412588 ]
 [25.194082 ]
 [24.76882  ]
 [27.106558 ]
 [26.641066 ]
 [23.367605 ]
 [26.087652 ]
 [27.639143 ]
 [16.02593  ]
 [20.953833 ]
 [17.73635  ]
 [16.481436 ]
 [19.23763  ]
 [16.658957 ]
 [13.868521 ]
 [15.593069 ]
 [17.995016 ]
 [17.838736 ]
 [14.623922 ]
 [16.645071 ]
 [15.99758  ]
 [23.11971  ]
 [17.422033 ]
 [15.582785 ]
 [24.083105 ]
 [24.47083  ]
 [27.213087 ]
 [25.618353 ]
 [26.614256 ]
 [26.018057 ]
 [23.523916 ]
 [26.069654 ]
 [26.896414 ]
 [26.406416 ]
 [25.988302 ]
 [25.195898 ]
 [24.980322 ]
 [17.13555  ]
 [13.911764 ]
 [18.500113 ]
 [15.586966 ]
 [17.45893  ]
 [15.138479 ]
 [17.461355 ]
 [17.760061 ]
 [15

We would like to see how good these predictions are.  We know what the correct MPG is for each car, so we can measure how close the neural network was.

In [10]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 5.769014358520508


We can also print out the first 10 cars, with predictions and actual MPG.

In [11]:
# Sample predictions
for i in range(10):
    print("{}. Car name: {}, MPG: {}, predicted MPG: {}".format(i+1,cars[i],y[i],pred[i]))

1. Car name: chevrolet chevelle malibu, MPG: 18.0, predicted MPG: [14.295172]
2. Car name: buick skylark 320, MPG: 15.0, predicted MPG: [14.286657]
3. Car name: plymouth satellite, MPG: 18.0, predicted MPG: [14.668158]
4. Car name: amc rebel sst, MPG: 16.0, predicted MPG: [16.209463]
5. Car name: ford torino, MPG: 17.0, predicted MPG: [15.1081295]
6. Car name: ford galaxie 500, MPG: 15.0, predicted MPG: [12.428854]
7. Car name: chevrolet impala, MPG: 14.0, predicted MPG: [12.156668]
8. Car name: plymouth fury iii, MPG: 14.0, predicted MPG: [12.6303005]
9. Car name: pontiac catalina, MPG: 14.0, predicted MPG: [13.063772]
10. Car name: amc ambassador dpl, MPG: 15.0, predicted MPG: [12.959219]


# Example of TensorFlow Classification: Iris

This is a very simple example of how to perform the Iris classification using TensorFlow. The iris.csv file is used. 


In [12]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


df=pd.read_csv("data/iris.csv",na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

In [13]:
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [17]:
x.shape

(150, 4)

In [14]:
y  #  This is one-hot encoding.  Only one value is 1.0 (hot)

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [15]:
y.shape

(150, 3)

In [16]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1], activation='softmax')) # Output

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
150/150 - 0s - loss: 1.7737
Epoch 2/100
150/150 - 0s - loss: 1.1822
Epoch 3/100
150/150 - 0s - loss: 0.9610
Epoch 4/100
150/150 - 0s - loss: 0.9741
Epoch 5/100
150/150 - 0s - loss: 0.9498
Epoch 6/100
150/150 - 0s - loss: 0.8761
Epoch 7/100
150/150 - 0s - loss: 0.8198
Epoch 8/100
150/150 - 0s - loss: 0.7843
Epoch 9/100
150/150 - 0s - loss: 0.7558
Epoch 10/100
150/150 - 0s - loss: 0.7187
Epoch 11/100
150/150 - 0s - loss: 0.6870
Epoch 12/100
150/150 - 0s - loss: 0.6624
Epoch 13/100
150/150 - 0s - loss: 0.6389
Epoch 14/100
150/150 - 0s - loss: 0.6154
Epoch 15/100
150/150 - 0s - loss: 0.5941
Epoch 16/100
150/150 - 0s - loss: 0.5752
Epoch 17/100
150/150 - 0s - loss: 0.5607
Epoch 18/100
150/150 - 0s - loss: 0.5396
Epoch 19/100
150/150 - 0s - loss: 0.5261
Epoch 20/100
150/150 - 0s - loss: 0.5097
Epoch 21/100
150/150 - 0s - loss: 0.4950
Epoch 22/100
150/150 - 0s - loss: 0.4830
Epoch 23/100
150/150 - 0s - loss: 0.4719
Epoch 24/100
150/150 - 0s - loss: 0.4604
Epoch 25/100
150/150 - 0s

In [21]:
# Print out number of species found:
print(species)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


Now that you have a neural network trained, we would like to be able to use it. There were 3 types of iris (Iris-setosa, Iris-versicolor, and Iris-virginica).  

In [17]:
pred = model.predict(x)
pred.shape

(150, 3)

In [18]:
pred

array([[9.94164884e-01, 5.83512057e-03, 2.32534738e-08],
       [9.86875892e-01, 1.31240366e-02, 1.11497279e-07],
       [9.91153419e-01, 8.84644222e-03, 8.72134578e-08],
       [9.86429691e-01, 1.35700777e-02, 1.95486635e-07],
       [9.94881392e-01, 5.11862244e-03, 2.24976127e-08],
       [9.94343758e-01, 5.65630849e-03, 1.79165767e-08],
       [9.91545498e-01, 8.45437124e-03, 1.10032879e-07],
       [9.92240906e-01, 7.75912683e-03, 4.21070361e-08],
       [9.83247697e-01, 1.67519767e-02, 3.79106780e-07],
       [9.88611281e-01, 1.13885617e-02, 7.96551873e-08],
       [9.95446980e-01, 4.55301348e-03, 9.47254808e-09],
       [9.90557075e-01, 9.44275316e-03, 8.06514890e-08],
       [9.88249421e-01, 1.17504681e-02, 9.91135920e-08],
       [9.90830779e-01, 9.16913245e-03, 1.63509270e-07],
       [9.98054266e-01, 1.94575964e-03, 1.48583368e-09],
       [9.98043299e-01, 1.95667427e-03, 2.14770113e-09],
       [9.96490180e-01, 3.50976014e-03, 8.51491588e-09],
       [9.93371725e-01, 6.62831

In [25]:
# print y.  
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

#### The column (pred) with the highest probability is  the prediction of the neural network.  

#### Use argmax function to find the index of the maximum prediction for each row.

In [19]:
# Usually the column (pred) with the highest prediction is considered to be the prediction of the neural network.  It is easy
# to convert the predictions to the expected iris species.  The argmax function finds the index of the maximum prediction
# for each row.

predict_classes = np.argmax(pred,axis=1)

true_classes = np.argmax(y,axis=1)

print("Predictions: {}".format(predict_classes))
print("True: {}".format(true_classes))

Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
True: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [27]:
# Of course it is very easy to turn these indexes back into iris species.  We just use the species list that we created earlier.
print(species[predict_classes[0:10]])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [20]:

#For all of the iris predictions, what percent were correct?  

correct = metrics.accuracy_score(true_classes, predict_classes)
print("Accuracy: {}".format(correct))

Accuracy: 0.98


The code below performs two ad hoc predictions.  

*** Remember x should be a 2D array! ***

In [21]:
# ad hoc prediction
sample_flower = np.array( [[5.0,3.0,4.0,2.0]], dtype=float)
pred = model.predict(sample_flower)
print(pred)

[[0.00240806 0.33512685 0.66246516]]


In [22]:
pred = np.argmax(pred, axis=1)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5. 3. 4. 2.]] is: ['Iris-virginica']


Notice that the argmax in the second prediction requires **axis=1**?  Since we have a 2D array now, we must specify which axis to take the argmax over.  The value **axis=1** specifies we want the max column index for each row.

In [23]:
# predict two sample flowers
sample_flower = np.array( [[5.0,3.0,4.0,2.0],[5.2,3.5,1.5,0.8]], dtype=float)
pred = model.predict(sample_flower)
print(pred)

[[2.4080558e-03 3.3512670e-01 6.6246527e-01]
 [9.8348677e-01 1.6512996e-02 2.3293094e-07]]


In [24]:
pred = np.argmax(pred, axis=1)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5.  3.  4.  2. ]
 [5.2 3.5 1.5 0.8]] is: ['Iris-virginica' 'Iris-setosa']


# Load/Save Trained Network

Complex neural networks will take a long time to fit/train.  It is helpful to be able to save these neural networks so that they can be reloaded later.  A reloaded neural network will not require retraining.  Keras provides three formats for neural network saving.

* **YAML** - Stores the neural network structure (no weights) in the [YAML file format](https://en.wikipedia.org/wiki/YAML).
* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Stores the complete neural network (with weights) in the [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 

Usually you will want to save in HDF5.

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data/"
save_path = "./dnn/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=0,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Before save score (RMSE): {}".format(score))


# save entire network to HDF5 (save everything)
model.save(os.path.join(save_path,"network.hdf5"))

Before save score (RMSE): 6.450832843780518


Now we reload the network and perform another prediction.  The RMSE should match the previous one exactly if the neural network was really saved and reloaded.

In [26]:
from tensorflow.keras.models import load_model

model2 = load_model(os.path.join(save_path,"network.hdf5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("After load score (RMSE): {}".format(score))

W0913 14:31:15.962442 14488 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0913 14:31:15.963423 14488 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


After load score (RMSE): 6.450832843780518


### References:

* [Google Colab](https://colab.research.google.com/) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow with free GPU support.  No setup needed.
* [IBM Cognitive Class Labs](https://www.datascientistworkbench.com) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow.  No setup needed.
* [Python Anaconda](https://www.continuum.io/downloads) - Python distribution that includes many data science packages, such as Numpy, Scipy, Scikit-Learn, Pandas, and much more.
* [TensorFlow](https://www.tensorflow.org/) - Google's mathematics package for deep learning.
* [Kaggle](https://www.kaggle.com/) - Competitive data science.  Good source of sample data.
* T81-558: Applications of Deep Neural Networks. Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/)